In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("C:/Users/debas/Downloads/bacnet_latest_data.csv")

In [ ]:
ahu1_df = df[df["equipment_id"] == "Ahu1"]

In [ ]:
ahu1_df["datapoint"].unique()

array(['PIDTR', 'SpCo2Hi', 'AlmPreFlt', 'SpCo2Band', 'SpMaxVFD', 'AlmFlw',
       'Co2RA2', 'TBHuSu', 'AlmNtAuto', 'SpFAD1', 'PIDVFD', 'SpMinVFD',
       'TBTRt', 'ChwTemp', 'TBCo2', 'TBTSu', 'HuR1', 'SpTRUnoc',
       'StaVFDSf', 'AlmTRLow', 'TBFbCHW', 'CMDSpdVFD', 'HuAvg1',
       'SpHuSpcCt', 'CmdVFD', 'TSOcc', 'AlmTRHi', 'TBTe', 'StaFlw',
       'StaAuto', 'FbFAD', 'CmdCHW', 'SpTREff', 'Co2RA', 'TempSp2',
       'AlmFAD', 'TBCMDVFD', 'HuR', 'TBCMDCHW', 'SpHuR', 'CmdFAD',
       'TrAvg', 'RtCo2', 'SpFAD', 'SpTRHiLo', 'TBStaFlw', 'AlmTrpSf',
       'AlmRACoHi', 'Almbag', 'TsOn', 'TRe', 'HuSu', 'HuR2', 'TSu',
       'TBFbFAD', 'Co2Avg', 'FbVFD', 'SpTROcc', 'TempSp1', 'FireAlm',
       'TBHuRt'], dtype=object)

In [ ]:
df["system_type"].unique()

array(['ROpUnDev(1)', 'ExFans', 'AHU', 'MtrEMU', 'RHvacCoo', 'HVAC',
       'RGrnLf', 'ROpModDtr', 'MAHU1', 'SMDB', 'SPLITUNIT'], dtype=object)

In [ ]:
ahu1_df[ahu1_df["datapoint"] == "SpTREff"]["monitoring_data"].unique()

array(['23', '20', '26', '21', '18', '22', '19'], dtype=object)

In [ ]:
import requests
import json
from datetime import datetime

def optimize_and_compare(df, target_variable="FbVFD", output_file="optimization_results_new.json", verbose=True):
    """
    Takes each row from df, sends to optimization endpoint, and saves results.
    
    Args:
        df: DataFrame with AHU data
        target_variable: The variable to optimize (default: "FbVFD")
        output_file: Path to save JSON results
        verbose: Show detailed error messages
    
    Returns:
        List of results with timestamp, actual, predicted, and difference
    """
    results = []
    endpoint = "http://127.0.0.1:8000/prod/generic_optimize"
    
    for idx, row in df.iterrows():
        try:
            # Convert timestamp to string for JSON serialization
            timestamp = str(row['timestamp']) if pd.notna(row['timestamp']) else str(idx)
            
            actual_value = float(row[target_variable]) if pd.notna(row[target_variable]) else None
            
            current_conditions = {}
            for col in df.columns:
                if pd.notna(row[col]):
                    current_conditions[col] = str(row[col])
                else:
                    current_conditions[col] = "0"
            
            request_body = {
                "current_conditions": current_conditions,
                "target_variable": target_variable,
                "optimization_method": "random",
                "n_iterations": 20
            }
            
            response = requests.post(endpoint, json=request_body, timeout=30)
            
            if response.status_code == 200:
                response_data = response.json()
                predicted_value = response_data.get("best_target_value", None)
                optimized_setpoints = response_data.get("best_setpoints", {})
                
                current_setpoints = {}
                for setpoint_name in optimized_setpoints.keys():
                    if setpoint_name in df.columns and pd.notna(row[setpoint_name]):
                        current_setpoints[setpoint_name] = float(row[setpoint_name])
                
                if actual_value is not None and predicted_value is not None:
                    difference = actual_value - predicted_value
                else:
                    difference = None
                
                result = {
                    "timestamp": timestamp,
                    "actual_value": actual_value,
                    "predicted_value": predicted_value,
                    "difference_actual_and_pred": difference,
                    "current_setpoints": current_setpoints,
                    "optimized_setpoints": optimized_setpoints
                }
                results.append(result)
                
                print(f"✓ Processed row {len(results)}/{len(df)} - Timestamp: {timestamp}, Predicted: {predicted_value}")
            else:
                error_msg = f"HTTP {response.status_code}"
                if verbose:
                    try:
                        error_detail = response.json()
                        error_msg = f"{error_msg} - {error_detail}"
                    except:
                        error_msg = f"{error_msg} - {response.text[:200]}"
                
                print(f"✗ Failed for timestamp {timestamp}: {error_msg}")
                results.append({
                    "timestamp": timestamp,
                    "actual_value": actual_value,
                    "predicted_value": None,
                    "difference_actual_and_pred": None,
                    "current_setpoints": {},
                    "optimized_setpoints": {},
                    "error": error_msg
                })
        
        except Exception as e:
            error_msg = str(e)
            if verbose:
                import traceback
                error_msg = traceback.format_exc()
            
            timestamp_str = str(row['timestamp']) if 'timestamp' in row and pd.notna(row['timestamp']) else str(idx)
            print(f"✗ Error processing timestamp {timestamp_str}: {error_msg}")
            results.append({
                "timestamp": timestamp_str,
                "actual_value": actual_value if 'actual_value' in locals() else None,
                "predicted_value": None,
                "difference_actual_and_pred": None,
                "current_setpoints": {},
                "optimized_setpoints": {},
                "error": error_msg
            })
    
    with open(output_file, 'w') as f:
        json.dump(results, f, indent=2)
    
    print(f"\n✓ Saved {len(results)} results to {output_file}")
    
    return results

print("Function 'optimize_and_compare()' created successfully!")

Function 'optimize_and_compare()' created successfully!


In [ ]:
for idx, row in df.iterrows():
    print(row['timestamp'])
    

2025-11-17 08:08:45.711000
2025-11-17 08:24:51.286000
2025-11-17 08:42:37.023000
2025-11-17 08:56:33.863000
2025-11-17 09:14:40.216000
2025-11-17 09:32:33.012000
2025-11-17 09:48:23.925000
2025-11-17 10:04:13.569000
2025-11-17 10:22:02.054000
2025-11-17 10:37:34.218000
2025-11-17 10:53:21.439000
2025-11-17 11:08:57.972000
2025-11-17 11:24:35.820000
2025-11-17 11:41:13.051000
2025-11-17 11:59:08.751000
2025-11-17 12:15:15.161000
2025-11-17 12:31:33.137000
2025-11-17 12:49:07.978000
2025-11-17 13:04:41.408000
2025-11-17 13:25:21.252000
2025-11-17 13:43:46.979000
2025-11-17 13:59:45.383000
2025-11-17 14:17:45.468000
2025-11-17 14:33:59.721000
2025-11-17 14:50:30.497000
2025-11-17 15:09:04.519000
2025-11-17 15:25:06.394000
2025-11-17 15:42:25.035000
2025-11-17 15:58:19.964000
2025-11-17 16:14:18.195000
2025-11-17 16:32:18.046000
2025-11-17 16:48:30.052000
2025-11-17 17:04:38.199000
2025-11-17 17:24:51.406000
2025-11-17 17:41:46.761000
2025-11-17 17:57:46.024000
2025-11-17 18:13:47.825000
2

In [ ]:
ahu1_df.rename(columns={"data_received_on": "timestamp"}, inplace=True)

C:\Users\debas\AppData\Local\Temp\ipykernel_10420\1432825279.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ahu1_df.rename(columns={"data_received_on": "timestamp"}, inplace=True)


In [ ]:
pivoted_table = ahu1_df.pivot_table(
    index="timestamp",
    columns="datapoint",
    values='monitoring_data',
    aggfunc='first'  
)

In [ ]:
pivoted_table.dropna(how='any', inplace=True)
pivoted_table

datapoint,AlmFAD,AlmFlw,AlmNtAuto,AlmPreFlt,AlmRACoHi,AlmTRHi,AlmTRLow,AlmTrpSf,Almbag,CMDSpdVFD,...,StaAuto,StaFlw,StaVFDSf,TRe,TSOcc,TSu,TempSp1,TempSp2,TrAvg,TsOn
timestamp,,,,,,,,,,,,,,,,,,,,,
2025-11-07T10:17:20.315 UTC,inactive,inactive,inactive,inactive,inactive,inactive,inactive,inactive,inactive,80,...,active,active,active,23.15999984741211,1,22.454999923706055,20.1299991607666,18.834999084472656,19.50749969482422,1
2025-11-07T10:33:47.450 UTC,inactive,inactive,inactive,inactive,inactive,inactive,inactive,inactive,inactive,80,...,active,active,active,23.184999465942383,1,22.5,20.489999771118164,19.299999237060547,19.959999084472656,1
2025-11-07T10:50:12.004 UTC,inactive,inactive,inactive,inactive,inactive,inactive,inactive,inactive,inactive,80,...,active,active,active,23.344999313354492,1,22.510000228881836,21.119998931884766,19.59000015258789,20.264999389648438,1
2025-11-07T11:06:56.469 UTC,inactive,inactive,inactive,inactive,inactive,inactive,inactive,inactive,inactive,80,...,active,active,active,23.385000228881836,1,22.600000381469727,21.170000076293945,19.81999969482422,20.5049991607666,1
2025-11-07T11:23:06.602 UTC,inactive,inactive,inactive,inactive,inactive,inactive,inactive,inactive,inactive,80,...,active,active,active,23.399999618530273,1,22.60999870300293,21.369998931884766,19.979999542236328,20.684999465942383,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-11-24T08:59:13.920 UTC,inactive,inactive,inactive,inactive,inactive,inactive,inactive,inactive,inactive,50,...,active,active,active,24.21500015258789,1,24.549999237060547,22.184999465942383,19.934999465942383,21.087499618530273,1
2025-11-24T09:15:00.088 UTC,inactive,inactive,inactive,inactive,inactive,inactive,inactive,inactive,inactive,50,...,active,active,active,24.279998779296875,1,24.59000015258789,22.314998626708984,19.98499870300293,21.172500610351562,1
2025-11-24T09:32:58.482 UTC,inactive,inactive,inactive,inactive,inactive,inactive,inactive,inactive,inactive,50,...,active,active,active,24.244998931884766,1,24.579999923706055,22.385000228881836,20.059999465942383,21.224998474121094,1


In [ ]:
pivoted_table.reset_index(inplace=True)
pivoted_table["timestamp"] = pd.to_datetime(pivoted_table["timestamp"])
pivoted_table.set_index("timestamp", inplace=True)

In [ ]:
pivoted_table.sort_index()

datapoint,AlmFAD,AlmFlw,AlmNtAuto,AlmPreFlt,AlmRACoHi,AlmTRHi,AlmTRLow,AlmTrpSf,Almbag,CMDSpdVFD,...,StaAuto,StaFlw,StaVFDSf,TRe,TSOcc,TSu,TempSp1,TempSp2,TrAvg,TsOn
timestamp,,,,,,,,,,,,,,,,,,,,,
2025-11-07 10:17:20.315000+00:00,inactive,inactive,inactive,inactive,inactive,inactive,inactive,inactive,inactive,80,...,active,active,active,23.15999984741211,1,22.454999923706055,20.1299991607666,18.834999084472656,19.50749969482422,1
2025-11-07 10:33:47.450000+00:00,inactive,inactive,inactive,inactive,inactive,inactive,inactive,inactive,inactive,80,...,active,active,active,23.184999465942383,1,22.5,20.489999771118164,19.299999237060547,19.959999084472656,1
2025-11-07 10:50:12.004000+00:00,inactive,inactive,inactive,inactive,inactive,inactive,inactive,inactive,inactive,80,...,active,active,active,23.344999313354492,1,22.510000228881836,21.119998931884766,19.59000015258789,20.264999389648438,1
2025-11-07 11:06:56.469000+00:00,inactive,inactive,inactive,inactive,inactive,inactive,inactive,inactive,inactive,80,...,active,active,active,23.385000228881836,1,22.600000381469727,21.170000076293945,19.81999969482422,20.5049991607666,1
2025-11-07 11:23:06.602000+00:00,inactive,inactive,inactive,inactive,inactive,inactive,inactive,inactive,inactive,80,...,active,active,active,23.399999618530273,1,22.60999870300293,21.369998931884766,19.979999542236328,20.684999465942383,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-11-24 08:59:13.920000+00:00,inactive,inactive,inactive,inactive,inactive,inactive,inactive,inactive,inactive,50,...,active,active,active,24.21500015258789,1,24.549999237060547,22.184999465942383,19.934999465942383,21.087499618530273,1
2025-11-24 09:15:00.088000+00:00,inactive,inactive,inactive,inactive,inactive,inactive,inactive,inactive,inactive,50,...,active,active,active,24.279998779296875,1,24.59000015258789,22.314998626708984,19.98499870300293,21.172500610351562,1
2025-11-24 09:32:58.482000+00:00,inactive,inactive,inactive,inactive,inactive,inactive,inactive,inactive,inactive,50,...,active,active,active,24.244998931884766,1,24.579999923706055,22.385000228881836,20.059999465942383,21.224998474121094,1


In [ ]:
needed_date = pd.to_datetime("2025-11-17 08:08:45.711")

In [ ]:
reset_table = pivoted_table.reset_index()
# Convert timezone-aware timestamps to naive for comparison
reset_table["timestamp"] = reset_table["timestamp"].dt.tz_localize(None)
df = reset_table[reset_table["timestamp"] >= needed_date]

In [ ]:
df.set_index("timestamp", inplace=True)

In [ ]:
df.reset_index(inplace=True)

In [ ]:
df

datapoint,timestamp,AlmFAD,AlmFlw,AlmNtAuto,AlmPreFlt,AlmRACoHi,AlmTRHi,AlmTRLow,AlmTrpSf,Almbag,...,StaAuto,StaFlw,StaVFDSf,TRe,TSOcc,TSu,TempSp1,TempSp2,TrAvg,TsOn
0,2025-11-17 08:08:45.711,inactive,inactive,inactive,inactive,inactive,inactive,inactive,inactive,inactive,...,active,active,active,25.064998626708984,1,22.35999870300293,22.174999237060547,21.135000228881836,21.654998779296875,1
1,2025-11-17 08:24:51.286,inactive,inactive,inactive,inactive,inactive,inactive,inactive,inactive,inactive,...,active,active,active,25.10999870300293,1,22.65999984741211,22.184999465942383,21.135000228881836,21.61750030517578,1
2,2025-11-17 08:42:37.023,inactive,inactive,inactive,inactive,inactive,inactive,inactive,inactive,inactive,...,active,active,active,24.045000076293945,1,15.414999961853027,21.06999969482422,21.34000015258789,21.119998931884766,1
3,2025-11-17 08:56:33.863,inactive,inactive,inactive,inactive,inactive,inactive,inactive,inactive,inactive,...,active,active,active,23.524999618530273,1,15.344999313354492,20.46500015258789,21.135000228881836,20.737499237060547,1
4,2025-11-17 09:14:40.216,inactive,inactive,inactive,inactive,inactive,inactive,inactive,inactive,inactive,...,active,active,active,22.91499900817871,1,15.38499927520752,19.65999984741211,20.875,20.23999786376953,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599,2025-11-24 08:59:13.920,inactive,inactive,inactive,inactive,inactive,inactive,inactive,inactive,inactive,...,active,active,active,24.21500015258789,1,24.549999237060547,22.184999465942383,19.934999465942383,21.087499618530273,1
600,2025-11-24 09:15:00.088,inactive,inactive,inactive,inactive,inactive,inactive,inactive,inactive,inactive,...,active,active,active,24.279998779296875,1,24.59000015258789,22.314998626708984,19.98499870300293,21.172500610351562,1
601,2025-11-24 09:32:58.482,inactive,inactive,inactive,inactive,inactive,inactive,inactive,inactive,inactive,...,active,active,active,24.244998931884766,1,24.579999923706055,22.385000228881836,20.059999465942383,21.224998474121094,1
602,2025-11-24 09:48:42.755,inactive,inactive,inactive,inactive,inactive,inactive,inactive,inactive,inactive,...,active,active,active,24.135000228881836,1,24.529998779296875,22.43000030517578,20.154998779296875,21.294998168945312,1


In [ ]:
# optimize_and_compare(df)

✓ Processed row 1/604 - Timestamp: 2025-11-17 08:08:45.711000, Predicted: 50.528446197509766
✓ Processed row 2/604 - Timestamp: 2025-11-17 08:24:51.286000, Predicted: 73.17650604248047
✓ Processed row 2/604 - Timestamp: 2025-11-17 08:24:51.286000, Predicted: 73.17650604248047
✓ Processed row 3/604 - Timestamp: 2025-11-17 08:42:37.023000, Predicted: 100.36717987060547
✓ Processed row 3/604 - Timestamp: 2025-11-17 08:42:37.023000, Predicted: 100.36717987060547
✓ Processed row 4/604 - Timestamp: 2025-11-17 08:56:33.863000, Predicted: 100.277587890625
✓ Processed row 4/604 - Timestamp: 2025-11-17 08:56:33.863000, Predicted: 100.277587890625
✓ Processed row 5/604 - Timestamp: 2025-11-17 09:14:40.216000, Predicted: 100.34867095947266
✓ Processed row 5/604 - Timestamp: 2025-11-17 09:14:40.216000, Predicted: 100.34867095947266
✓ Processed row 6/604 - Timestamp: 2025-11-17 09:32:33.012000, Predicted: 100.46564483642578
✓ Processed row 6/604 - Timestamp: 2025-11-17 09:32:33.012000, Predicted: 10

[{'timestamp': '2025-11-17 08:08:45.711000',
  'actual_value': 51.119998931884766,
  'predicted_value': 50.528446197509766,
  'difference_actual_and_pred': 0.591552734375,
  'current_setpoints': {'SpMinVFD': 50.0, 'SpTREff': 22.0, 'SpTROcc': 22.0},
  'optimized_setpoints': {'SpMinVFD': 65.0, 'SpTREff': 23.0, 'SpTROcc': 20.0}},
 {'timestamp': '2025-11-17 08:24:51.286000',
  'actual_value': 51.15999984741211,
  'predicted_value': 73.17650604248047,
  'difference_actual_and_pred': -22.01650619506836,
  'current_setpoints': {'SpMinVFD': 50.0, 'SpTREff': 20.0, 'SpTROcc': 20.0},
  'optimized_setpoints': {'SpMinVFD': 25.0, 'SpTREff': 22.5, 'SpTROcc': 21.5}},
 {'timestamp': '2025-11-17 08:42:37.023000',
  'actual_value': 100.87999725341797,
  'predicted_value': 100.36717987060547,
  'difference_actual_and_pred': 0.5128173828125,
  'current_setpoints': {'SpMinVFD': 50.0, 'SpTREff': 20.0, 'SpTROcc': 20.0},
  'optimized_setpoints': {'SpMinVFD': 60.0, 'SpTREff': 21.5, 'SpTROcc': 20.0}},
 {'timesta